<a href="https://colab.research.google.com/github/SIVARAM7744/Shadowfox-/blob/main/SFintermediate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ShadowFox intermediate project - Loan approval prediction**

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import joblib
from google.colab import drive

In [2]:
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
loanexist = pd.read_csv('/content/drive/MyDrive/Shadowfox/intermediate/loan_prediction.csv')
columninsheet = [
    'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
    'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'
]


In [5]:
loanexist = loanexist[columninsheet]
for col in ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']:
    loanexist[col].fillna(loanexist[col].mode()[0], inplace=True)

loanexist.fillna(loanexist.mean(numeric_only=True), inplace=True)

<ipython-input-5-48896f2e5d6f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loanexist[col].fillna(loanexist[col].mode()[0], inplace=True)
<ipython-input-5-48896f2e5d6f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loanexist.fillna(loanexist.mean(numeric_only=True), inplace=True)


In [6]:
labellists = {}
for feature in ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']:
    encoder = LabelEncoder()
    loanexist[feature] = encoder.fit_transform(loanexist[feature])
    labellists[feature] = encoder


In [7]:
X = loanexist.drop('Loan_Status', axis=1)
y = loanexist['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = RandomForestClassifier()
model.fit(X_train, y_train)

joblib.dump(model, '/content/drive/MyDrive/Shadowfox/intermediate/loan_model.pkl')
joblib.dump(labellists, '/content/drive/MyDrive/Shadowfox/intermediate/labellists.pkl')

print("model training is completed and saved")

model training is completed and saved


In [10]:
import pandas as pd
import joblib
from google.colab import drive

In [11]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
pathh = '/content/drive/MyDrive/Shadowfox/intermediate/loan_model.pkl'
enc_path = '/content/drive/MyDrive/Shadowfox/intermediate/labellists.pkl'

In [19]:
model = joblib.load(pathh)
labellists = joblib.load(enc_path)

In [21]:
newloan = pd.read_csv('/content/drive/MyDrive/Shadowfox/intermediate/loan_prediction_check1.csv')
oridata = newloan.copy()
colavail = [
    'Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
    'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area'
]
newloan = newloan[colavail]

In [22]:
for col in ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']:
    newloan[col].fillna(newloan[col].mode()[0], inplace=True)
newloan.fillna(newloan.mean(numeric_only=True), inplace=True)

In [23]:
for feature in ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']:
    encoder = labellists[feature]
    unseen_labels = set(newloan[feature]) - set(encoder.classes_)
    if unseen_labels:
        newloan[feature] = newloan[feature].apply(lambda x: x if x in encoder.classes_ else encoder.classes_[0])
    newloan[feature] = encoder.transform(newloan[feature])
preds = model.predict(newloan)
oridata['Predicted_Loan_Status'] = preds
print("Predictions:")
print(oridata.head())

Predictions:
    Loan_ID  Gender Married  Dependents     Education Self_Employed  \
0  LP002949  Female      No           3      Graduate           NaN   
1  LP002950    Male     Yes           0  Not Graduate           NaN   
2  LP002953    Male     Yes           3      Graduate            No   
3  LP002958    Male      No           0      Graduate            No   
4  LP002959  Female     Yes           1      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0              416              41667       350.0               180   
1             2894               2792       155.0               360   
2             5703                  0       128.0               360   
3             3676               4301       172.0               360   
4            12000                  0       496.0               360   

   Credit_History Property_Area Predicted_Loan_Status  
0             NaN         Urban                     N  
1             1.0    